<a href="https://colab.research.google.com/github/kumaramardeep342/Colab-Work/blob/main/Ed_AI__NLP_M4_CaseStudy2_wine_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study
The data for this assignment is taken from Kaggle. (https://www.kaggle.com/zynicide/wine-reviews)

This is a subset of the original data which contains information such as Country ,Description , Points and Designation of certain types of Wines . You have to use the perform the following functions on this file

 - Read the CSV file“Wine.csv” subset of  "winemag-data_first150k.csv"
 - Using Pre-process File which you created in module 2 case study 1, call the ‘Refine’ functionand get the pre-processed text for each ‘description’ in the csv file. Store it in a column named “Refined-Description”
 - Using ‘CountVectorization’ function from ‘Vectorization’ python file createdin case study 1of this module,vectorise all the rows in ‘Refined-Description’ column which you created in the above step. Store them in a column named “CountVectorizer”
 - Using ‘TF-IDFVectorization’functionfrom ‘Vectorization’ python file createdin case study 1 of this module,vectorise all the rows in ‘Refined-Description’ column and store the results in ‘TF-IDF Vectorizer Column”
 - Save changes to the CSV file.

 NOTE: We have created 3 Python Files till now
  - 1.PreProcess:Has functions to tokenize, remove stop words & lemmatize string
  -  2.Corpus: Takes multiples strings/documents and returns a corpus
  - 3.Vectorization:Has functions for presence absence, count and tf-idf
  vectorization
  
  These will be helpful in upcoming case studies and projects.


# Summary - wine-reviews
0. Load the dataset
1. Import libraries
2. Read the dataset
3. Data Preprocessing on X_data[Countvectorizer , TFIDFVectorizer] and Y_data[label encoding]
4. Split the dataset into X_data and Y_data
* ---------------------- General Add on -----------------------------
5. Split the data into training and testing - 80% training and 20% testing
6. Use only 80% training data to train the model[MultinominalNB]
7. Hyperparameter tunning
8. pass the test to the trained model to predict y_pred_test - validation metrics
9. Create a samml API - x_data[response] - post - "airfare" etc.

# Load the dataset

In [28]:
! pip install -q opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/zynicide/wine-reviews')

Skipping, found downloaded files in "./wine-reviews" (use force=True to force download)


#  About The Dataset & Model

- The dataset offers some great opportunities for sentiment analysis and other text related predictive models.

- The model still won't be able to taste the wine, but theoretically it could identify the wine based on a description.

- The goal is to create a model that can identify the variety, winery, and location of a wine based on a description.

- It is having total 14  features. If we take -  description as input features then possible output variable status :
- Unnamed: 0 - NA , country - Yes, designation - Yes ,points - Yes , price - Yes , province - Yes, region_1 - Yes, region_2 - Yes, variety - Yes, winery - Yes, title - NA, tasters name - NA, taster's Twitter handle - NA

- In this model  - model will  predict wine variety based on description.

# 1.Import libraries

In [29]:
#enable table format
from google.colab import data_table
data_table.enable_dataframe_formatter()

#disable table format
# from google.colab import data_table
# data_table.disable_dataframe_formatter()

import pandas as pd  # for reading the dataset
import numpy as np  # for mathematical operations

#Pre-Processing
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer

# import sys
# # Add the directory to the search path
# sys.path.append('/content/drive/MyDrive/Colab Work/Edureka AI Certification/Re-Usable Code/')

# # Now import the module. This is mine created library
# from PreProcess import *
# from Corpus import *
# from Vectorization import *


#Vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Label Encoding
from sklearn.preprocessing import LabelEncoder
#model training
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2.Read the dataset

In [30]:
data = pd.read_csv('/content/wine-reviews/winemag-data_first150k.csv')
df = data[['Unnamed: 0','winery','country','points','description','designation','variety']][0:2000]
#df = data[['Unnamed: 0','winery','country','points','description','designation','variety']]
df.head()

,Unnamed: 0,winery,country,points,description,designation,variety
0,0,Heitz,US,96,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,Cabernet Sauvignon
1,1,Bodega Carmen Rodríguez,Spain,96,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,Tinta de Toro
2,2,Macauley,US,96,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,Sauvignon Blanc
3,3,Ponzi,US,96,"This spent 20 months in 30% new French oak, an...",Reserve,Pinot Noir
4,4,Domaine de la Bégude,France,95,"This is the top wine from La Bégude, named aft...",La Brûlade,Provence red blend


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   2000 non-null   int64 
 1   winery       2000 non-null   object
 2   country      1998 non-null   object
 3   points       2000 non-null   int64 
 4   description  2000 non-null   object
 5   designation  1498 non-null   object
 6   variety      2000 non-null   object
dtypes: int64(2), object(5)
memory usage: 109.5+ KB


In [32]:
#len(df.variety.value_counts())
#df.variety.value_counts().values
df.variety.value_counts()

variety
Pinot Noir                  230
Chardonnay                  170
Cabernet Sauvignon          149
Red Blend                   134
Bordeaux-style Red Blend     96
                           ... 
Blauburgunder                 1
St. Laurent                   1
Nero d'Avola                  1
Carignane                     1
Rosado                        1
Name: count, Length: 165, dtype: int64

>Findings
- There are 2000 entries and 165 different type of different wine variety

In [33]:
#considerding those brand varieties which got repeated at least 10 times.
df.variety.value_counts()[df.variety.value_counts().values>9]
# len(df.variety.value_counts()[df.variety.value_counts().values>9])

variety
Pinot Noir                    230
Chardonnay                    170
Cabernet Sauvignon            149
Red Blend                     134
Bordeaux-style Red Blend       96
Riesling                       89
Nebbiolo                       86
Sauvignon Blanc                72
Portuguese Red                 72
Syrah                          66
Sangiovese                     59
Zinfandel                      54
Rosé                           45
Malbec                         44
White Blend                    43
Tempranillo                    36
Merlot                         30
Pinot Gris                     25
Sparkling Blend                23
Portuguese White               21
Rhône-style Red Blend          20
Tempranillo Blend              20
Cabernet Franc                 20
Grenache                       16
Bordeaux-style White Blend     16
Rhône-style White Blend        13
Gewürztraminer                 13
Viognier                       11
Barbera                        10
Name: 

In [34]:
df.variety.value_counts()[df.variety.value_counts().values>9].keys()

Index(['Pinot Noir', 'Chardonnay', 'Cabernet Sauvignon', 'Red Blend',
       'Bordeaux-style Red Blend', 'Riesling', 'Nebbiolo', 'Sauvignon Blanc',
       'Portuguese Red', 'Syrah', 'Sangiovese', 'Zinfandel', 'Rosé', 'Malbec',
       'White Blend', 'Tempranillo', 'Merlot', 'Pinot Gris', 'Sparkling Blend',
       'Portuguese White', 'Rhône-style Red Blend', 'Tempranillo Blend',
       'Cabernet Franc', 'Grenache', 'Bordeaux-style White Blend',
       'Rhône-style White Blend', 'Gewürztraminer', 'Viognier', 'Barbera'],
      dtype='object', name='variety')

In [35]:
top_29_varieties = df.variety.value_counts()[df.variety.value_counts().values>9].keys()
wine =[]
for idx,var in enumerate(df.variety):
  if var in top_29_varieties:
    wine.append([df['description'][idx],df['variety'][idx]])
df_wine = pd.DataFrame(wine,columns=['description','variety'])
df_wine.head()

,description,variety
0,This tremendous 100% varietal wine hails from ...,Cabernet Sauvignon
1,Mac Watson honors the memory of a wine once ma...,Sauvignon Blanc
2,"This spent 20 months in 30% new French oak, an...",Pinot Noir
3,This re-named vineyard was formerly bottled as...,Pinot Noir
4,The producer sources from two blocks of the vi...,Pinot Noir


In [36]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  1683 non-null   object
 1   variety      1683 non-null   object
dtypes: object(2)
memory usage: 26.4+ KB


# 3.Data Preprocessing on X_data and Y_data and 4. Split the dataset into X_data and Y_data
Reduce the model complexity such a way accuracy should improve withoutoverfitting or underfitting.

### Refined-Description

In [37]:
#Using Pre-process File which you created in module 2 case study 1, call the ‘Refine’ functionand get the pre-processed text for each ‘description’ in the csv file. Store it in a column named “Refined-Description”

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def remove_stop_words(text):
    stop_words = stopwords.words('english')
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    stop_words_text = [word for word in words if word.lower() in stop_words]
    return  ' '.join(filtered_words) ,  stop_words_text

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def refine(text):
    tokens = tokenize(text)
    filtered_words, stop_words_text = remove_stop_words(text)
    lemmatized_words = lemmatize(filtered_words)
    return lemmatized_words

for i in range (len(df_wine)):
  df_wine['description'][i] = refine(df_wine['description'][i])


In [38]:
#help(df_wine.rename)
df_wine.rename(columns={'description': 'Refined-Description'}, inplace=True)
df_wine.head()

,Refined-Description,variety
0,tremendous 100 % varietal wine hail Oakville a...,Cabernet Sauvignon
1,Mac Watson honor memory wine made mother treme...,Sauvignon Blanc
2,"spent 20 month 30 % new French oak , incorpora...",Pinot Noir
3,re-named vineyard formerly bottled deLancellot...,Pinot Noir
4,producer source two block vineyard wine—one hi...,Pinot Noir


In [39]:
#help(df_wine.to_csv)
df_wine.to_csv("/content/drive/MyDrive/Colab Work/Edureka AI Certification/Dataset/Kaggle - Wine Reviews/df_wine_Refined.csv")

### Count Vectorization

In [40]:
#Using ‘CountVectorization’ function from ‘Vectorization’ python file createdin case study 1of this module,vectorise all the rows in ‘Refined-Description’ column which you created in the above step. Store them in a column named “CountVectorizer”

##  stopwords and word should repeat minimum 2 times
cv_nostop_min2_max= CountVectorizer(ngram_range =(1,1),
                #stop_words ="english",  ##stopwords considered as english
                max_df =0.98, #remove a word which repeated 98% in the documents
                min_df =2, # remove a word which  repeated atleast 2 times
              max_features =None )
X_nostop_min2_max= cv_nostop_min2_max.fit_transform(df_wine['Refined-Description'])
X_nostop_min2_max.shape
#X_nostop_min2_max.toarray()

(1683, 2505)

In [41]:
df_count = pd.DataFrame(X_nostop_min2_max.toarray(),columns=cv_nostop_min2_max.get_feature_names_out())
df_count.head()

,10,100,11,12,13,14,15,16,17,18,...,youth,youthful,youthfully,zealand,zest,zesty,zin,zinfandel,zingy,zippy
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_count.to_csv("/content/drive/MyDrive/Colab Work/Edureka AI Certification/Dataset/Kaggle - Wine Reviews/df_CountVectorizer.csv")

### TF-IDFVectorization

In [43]:
#Using ‘TF-IDFVectorization’functionfrom ‘Vectorization’ python file createdin case study 1 of this module,vectorise all the rows in ‘Refined-Description’ column and store the results in ‘TF-IDF Vectorizer Column”
### TF-IDF Vectorizer
vectorize_tfidf=TfidfVectorizer(ngram_range =(1,1),
                                #stop_words="english",   ##stopwords considered as english
                                max_df=0.98, #remove a word which repeated 98% in the documents
                                min_df=1, # remove a word no repeeation
                                max_features=None)
vectorize_tfidf.fit(df_wine['Refined-Description'])
X_tfidf=vectorize_tfidf.transform(df_wine['Refined-Description'])
X_tfidf.shape
#X_tfidf.toarray()

(1683, 4552)

In [44]:
df_tf_idf = pd.DataFrame(X_tfidf.toarray(),columns=vectorize_tfidf.get_feature_names_out())
df_tf_idf.head()

,10,100,11,12,122,13,14,15,15g,16,...,zestier,zesty,zin,zinfandel,zinfandels,zing,zingy,zins,zippy,émilion
0,0.0,0.153895,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.248195,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_tf_idf.to_csv("/content/drive/MyDrive/Colab Work/Edureka AI Certification/Dataset/Kaggle - Wine Reviews/df_TF-IDF Vectorizer.csv")

### Label Encoding

In [46]:

le=LabelEncoder()
y_data=le.fit_transform(df_wine['variety'])
y_data

array([ 4, 21, 12, ..., 25, 10, 12])

# 5.Split the data into training and testing and 6. Use only training data to train the model

In [47]:
# training on Count Vectorization
X_train,X_test,y_train,y_test=train_test_split(X_nostop_min2_max,
                                         y_data,test_size=0.2,
                                         random_state=42)

mnb = MultinomialNB()
mnb.fit(X_train,y_train)
y_pred_test=mnb.predict(X_test)
y_pred_test

array([20, 13, 12, 15, 23, 15, 12,  1, 12,  5, 15, 10,  5, 12,  5, 10, 20,
        5,  5,  1, 28,  4, 15,  5, 13, 12, 15,  4, 13,  5, 15, 27, 15, 18,
        4,  5, 12, 18,  4,  5, 15, 12, 20, 15, 13, 13,  4, 15,  5,  5, 13,
        4,  5,  4, 10, 13, 28, 12, 18,  1,  4, 12,  1, 10, 12,  4,  4, 15,
        4, 15, 12,  4, 10, 12, 12, 12, 10,  4, 15, 12,  1, 21, 12,  4, 23,
        1, 20,  1,  5,  5, 15,  4,  5,  5, 12,  4,  1,  4, 18, 15, 13,  4,
       21,  5, 15, 18, 10, 12, 18, 12, 10,  5, 10,  5,  4, 10, 15, 15, 18,
       18, 12, 15, 18,  1,  4,  5,  4, 13,  1, 12, 10,  4, 12,  5, 12, 12,
       18, 12,  4, 12,  5, 12,  4,  5, 12, 12,  1, 20, 13,  1, 12, 12, 15,
       10,  4, 15, 15, 12, 12, 21,  4, 10,  5,  4, 12, 15, 13, 13,  4, 10,
       13,  5, 13, 15, 12,  4, 15, 12, 10,  1, 10,  5,  5,  1, 15, 13,  5,
       12, 18, 12, 10, 12,  5, 12, 12, 12,  5, 12, 12, 15, 10, 20, 12, 15,
       12,  1, 12,  4, 21, 12, 13,  5, 12, 12, 12, 10, 15,  5, 20, 12, 21,
       15,  1,  5, 21, 12

# 8.pass the test to the trained model to predict y_pred_test - validation metrics

In [48]:
def model_accuracy(y_test,y_pred_test):
  cm= confusion_matrix(y_test,y_pred_test)
  accuracy = accuracy_score(y_test,y_pred_test)
  #print(cm)
  print(accuracy)

In [49]:
model_accuracy(y_test,y_pred_test)

0.5370919881305638


In [50]:
#X_tfidf -- model using TF & IDF vectorization
X_train,X_test,y_train,y_test=train_test_split(X_tfidf,
                                               y_data,test_size=0.2,
                                               random_state=42)

mnb = MultinomialNB()
mnb.fit(X_train,y_train)
y_pred_test=mnb.predict(X_test)
y_pred_test

array([15, 12, 12, 15, 12,  4, 12, 12, 12,  5, 15, 12,  5, 12,  5, 10, 15,
        5,  5, 12, 12,  4, 12,  5, 12, 12, 12, 12, 12,  5, 12,  5, 12,  5,
       12,  5, 12,  5, 12, 12, 12, 12, 10, 15, 12, 12, 12, 15,  5,  5,  1,
        4,  5, 12, 10, 12, 12, 12, 12, 12, 12, 12,  1, 15, 12, 12, 12,  4,
        4, 15, 12, 12, 15, 12, 12, 12, 10,  4, 12, 12,  1, 12, 12,  4,  4,
       12, 15, 12,  5,  5, 15, 12,  5,  5, 12, 12,  1, 12, 12, 12, 12,  4,
       12,  5, 15, 12, 10, 12,  5, 12, 10,  5, 10,  5,  4, 10, 15, 15,  5,
        5, 12, 15,  5, 12, 12,  5, 12, 12, 12, 12, 10, 12, 12, 12, 12, 12,
       12, 12,  4, 12,  5, 12, 12,  5, 12, 12, 12, 15, 12, 12, 12, 12, 15,
       10, 12, 12, 12, 12, 12,  5,  4, 10,  5,  4, 12, 12, 12,  5,  4, 10,
       12,  5, 12, 15, 12, 12, 15, 12, 10, 12, 12,  5,  5,  1, 12, 12, 12,
       12,  5, 12, 10, 12,  5, 12, 12, 12,  5, 12, 12, 12, 15, 15, 12, 15,
       12, 12, 12, 12, 12, 12, 12,  5, 12, 12, 12, 15, 12, 12, 15, 12,  5,
       12, 12,  5,  5, 12

In [51]:
model_accuracy(y_test,y_pred_test)

0.3798219584569733
